<a href="https://colab.research.google.com/github/MalikArslanBashir/Data-Science-Tasks/blob/main/lec07_II_solvedbyarslan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Wrangling and Regex

Working with text: applying string methods and regular expressions

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import zipfile
import pandas as pd

## Demo 1: Canonicalizing County Names

Load both **county_and_state.csv** and **county_and_population.csv**

In [ ]:
#code Here
state = pd.read_csv(r'C:\Users\dell\Downloads\county_and_state.csv')
population = pd.read_csv(r'C:\Users\dell\Downloads\county_and_population.csv')


# display both frames, display allows us to view a DataFrame without returning it as an object
#Code Here
state


,County,State
0,De Witt County,IL
1,Lac qui Parle County,MN
2,Lewis and Clark County,MT
3,St John the Baptist Parish,LS


In [ ]:
population

,County,Population
0,DeWitt,16798
1,Lac Qui Parle,8067
2,Lewis & Clark,55716
3,St. John the Baptist,43044


Both of these DataFrames share a "County" column. Unfortunately, formatting differences mean that we can't directly merge the two DataFrames using the "County"s.

In [ ]:
#merge both frames
merged_df = pd.merge(state, population, left_on='County', right_on='County')
merged_df

,County,State,Population


To address this, we can **canonicalize** the "County" string data to apply a common formatting.

In [ ]:
def canonicalize_county(county_series):
 return (county_series.str.lower().str.replace(' ', '').str.replace('&', 'and').str.replace('.', '').str.replace('county', '').str.replace('parish', ''))
                                                                        # remove space
                                                                          # replace &
                                                                            # remove dot
                                                                            # remove "county"
                                                                             # remove "parish"


Apply canonicalize_county on 'County' columns in both frames.

In [ ]:
population['County'] =canonicalize_county(population['County'])
state['County'] =canonicalize_county(state['County'])

# Code Here
print(population)
state

             County  Population
0            dewitt       16798
1       lacquiparle        8067
2     lewisandclark       55716
3  stjohnthebaptist       43044


C:\Users\dell\AppData\Local\Temp\ipykernel_7336\3853979773.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  return (county_series.str.lower().str.replace(' ', '').str.replace('&', 'and').str.replace('.', '').str.replace('county', '').str.replace('parish', ''))


,County,State
0,dewitt,IL
1,lacquiparle,MN
2,lewisandclark,MT
3,stjohnthebaptist,LS


Now, the merge works as expected!

In [ ]:
# code Here
merge_df = pd.merge(state, population, left_on='County', right_on='County')
merge_df


,County,State,Population
0,dewitt,IL,16798
1,lacquiparle,MN,8067
2,lewisandclark,MT,55716
3,stjohnthebaptist,LS,43044


## Demo 2: Extracting Log Data

Load log.txt

In [ ]:
import re
pd.read_table(r'C:\Users\dell\Downloads\log.txt', delimiter='/t', engine='python')

,"169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] ""GET /stat141/Winter04/ HTTP/1.1"" 200 2585 ""http://anson.ucdavis.edu/courses/"""
0,"193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] ""..."
1,"169.237.46.240 - """" [3/Feb/2006:10:18:37 -0800..."


Suppose we want to extract the day, month, year, hour, minutes, seconds, and timezone. Looking at the data, we see that these items are not in a fixed position relative to the beginning of the string. That is, slicing by some fixed offset isn't going to work.

In [ ]:
# code here
logs = []
with open(r'C:\Users\dell\Downloads\log.txt') as logs_file:
  for i, line in enumerate(logs_file):
    logs.append(line)

In [ ]:
logs

['169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n',
 '193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] "GET /stat141/Notes/dim.html HTTP/1.0" 404 302 "http://eeyore.ucdavis.edu/stat141/Notes/session.html"\n',
 '169.237.46.240 - "" [3/Feb/2006:10:18:37 -0800] "GET /stat141/homework/Solutions/hw1Sol.pdf HTTP/1.1"\n']

In [ ]:
len(logs)

3

In [ ]:
logs[0]

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

In [ ]:
# code here
#with string slicing
for i in range(0, len(logs)):
  print(logs[i][20:46])

26/Jan/2014:10:47:58 -0800
/Feb/2005:17:23:6 -0800] "
[3/Feb/2006:10:18:37 -0800


In [ ]:
#with built in split
# code here
for i in range(0, len(logs)):
  print(logs[i].split('[')[1].split(']')[0])

26/Jan/2014:10:47:58 -0800
2/Feb/2005:17:23:6 -0800
3/Feb/2006:10:18:37 -0800


Instead, we'll need to use some more sophisticated thinking. Let's focus on only the first line of the file.

In [ ]:
# code here

Apply string functions of python to extract date from first entry in log file.

In [ ]:
# find the text enclosed in square brackets
# split up the date/month/year
# split up the hour:minute:second
# split the timezone after the blank space

#with built in split
# code here
for i in range(0, len(logs)):
  day, month, other = logs[i].split('[')[1].split(']')[0].split('/')
  year, hour, min, other = other.split(':')
  sec, tz = other.split(' ')
  print(day, month, year, hour, min, sec, tz)


26 Jan 2014 10 47 58 -0800
2 Feb 2005 17 23 6 -0800
3 Feb 2006 10 18 37 -0800


This worked, but felt fairly "hacky" – the code above isn't particularly elegant. A much more sophisticated but common approach is to extract the information we need using a *regular expression*.


# Regular Expressions


## String Extraction with Regex

Python `re.findall` returns a list of all extracted matches: extract numbers from given string.

In [ ]:
import re

text = "My social security number is 123-45-6789 bro, or actually maybe it’s 321-45-6789.";
# code Here
re.findall(r'[0-9]+', text)

['123', '45', '6789', '321', '45', '6789']

<br/>

Now, let's see vectorized extraction in `pandas`:

 `.str.findall` returns a `Series` of lists of all matches in each record.

In [ ]:
import pandas as pd
data = ['987-65-4321','forty','123-45-6789 bro or 321-45-6789','999-99-9999']
# code here
df = pd.DataFrame(data)

In [ ]:
df[0].str.findall(r'[0-9]+')

0                   [987, 65, 4321]
1                                []
2    [123, 45, 6789, 321, 45, 6789]
3                   [999, 99, 9999]
Name: 0, dtype: object

Find all entries containing numbers in dataframe.

In [ ]:
# -> Series of lists
# code Here
df[~(df[0].str.isalnum())]

,0
0,987-65-4321
2,123-45-6789 bro or 321-45-6789
3,999-99-9999


## Extraction Using Regex Capture Groups

The Python function `re.findall`, in combination with parentheses returns specific substrings (i.e., **capture groups**) within each matched string, or **match**.

In [ ]:
text = """I will meet you at 08:30:00 pm tomorrow"""
# code here
matches = re.findall(r'([0-9]{2}):([0-9]{2}):([0-9]{2})', text)

In [ ]:
# the three capture groups in the first matched string
hour, minute, second = matches[0]
print(hour, minute, second)

08 30 00


<br/>

In `pandas`, we can use `.str.extract` to extract each capture group of **only the first match** of each record into separate columns.

In [ ]:
# back to SSNs
df_ssn

,SSN
0,987-65-4321
1,forty
2,123-45-6789 bro or 321-45-6789
3,999-99-9999


In [ ]:
# Will extract the first match of all groups
# code here
sns_df = pd.DataFrame(df[0].str.findall(r"(\d+)-(\d+)-(\d+)").explode()).dropna()
sns_df.columns=["sns"]
sns_df

,sns
0,"(987, 65, 4321)"
2,"(123, 45, 6789)"
2,"(321, 45, 6789)"
3,"(999, 99, 9999)"


In [ ]:
sns_df.sns.apply(pd.Series)

,0,1,2
0,987,65,4321
2,123,45,6789
2,321,45,6789
3,999,99,9999


Alternatively, `.str.extractall` extracts **all matches** of each record into separate columns. Rows are then MultiIndexed by original record index and match index.

In [ ]:
# -> DataFrame, one row per match
# code Here
df[0].str.extractall(r'(\d{3})-(\d{2})-(\d{4})')

0   1     2
  match               
0 0      987  65  4321
2 0      123  45  6789
  1      321  45  6789
3 0      999  99  9999

## Canonicalization with Regex

In regular Python, canonicalize with `re.sub` (standing for "substitute"):

In [ ]:
text = '<div><td valign="top">Moo</td></div>'
pattern = r"<[^>]+>"
#code here
re.sub(pattern, '', text)

'Moo'

In [ ]:
pattern = r'(\w+)=["\']?([0-9A-Za-z]+)["\']?\w*'
prop_val = re.findall(pattern, text)
prop_val

[('valign', 'top')]

<br/>

In `pandas`, canonicalize with `Series.str.replace`.

In [ ]:
# example dataframe of strings, convert in dataframe
df_html = ['<div><td valign="top">Moo</td></div>',
                   '<a href="http://ds100.org">Link</a>',
                   '<b>Bold text</b>']
print(df_html)

['<div><td valign="top">Moo</td></div>', '<a href="http://ds100.org">Link</a>', '<b>Bold text</b>']


In [ ]:
# Series -> Series
#Extract only words
pattern = r'<[^>]+>([\w\s]+)<[^>]+>'



# Revisiting Text Log Processing using Regex

### Python `re` version

In [ ]:
line = log_lines[0]
display(line)
# code here

'169.237.46.168 - - [26/Jan/2014:10:47:58 -0800] "GET /stat141/Winter04/ HTTP/1.1" 200 2585 "http://anson.ucdavis.edu/courses/"\n'

('26', 'Jan', '2014', '10', '47', '58', '-0800')

### `pandas` version

In [ ]:
# code here
df = pd.DataFrame(logs)
df

,0
0,169.237.46.168 - - [26/Jan/2014:10:47:58 -0800...
1,"193.205.203.3 - - [2/Feb/2005:17:23:6 -0800] ""..."
2,"169.237.46.240 - """" [3/Feb/2006:10:18:37 -0800..."


Option 1: `Series.str.findall`

In [ ]:
# code here
df[0].str.findall(r'\[(.+)\]')

0    [26/Jan/2014:10:47:58 -0800]
1      [2/Feb/2005:17:23:6 -0800]
2     [3/Feb/2006:10:18:37 -0800]
Name: 0, dtype: object

<br/>

Option 2: `Series.str.extractall`

In [ ]:
# code here
df_extracted = df[0].str.extractall(r'\[(\d{0,2})/(\w+)/(\d{4}):(\d{0,2}):(\d{0,2}):(\d{0,2})\s([-\d]+)\]')
df_extracted.columns = ["Day", "Month", "Year", "Hour", "Minute", "Second", "Time Zone"]
df_extracted = df_extracted.reset_index().drop(["level_0", "match"], axis=1)

Wrangling either of these two DataFrames into a nice format (like below) is left as an exercise for you!


||Day|Month|Year|Hour|Minute|Second|Time Zone|
|---|---|---|---|---|---|---|---|
|0|26|Jan|2014|10|47|58|-0800|
|1|2|Feb|2005|17|23|6|-0800|
|2|3|Feb|2006|10|18|37|-0800|


In [ ]:
# your code here
df_extracted

,Day,Month,Year,Hour,Minute,Second,Time Zone
0,26,Jan,2014,10,47,58,-0800
1,2,Feb,2005,17,23,6,-0800
2,3,Feb,2006,10,18,37,-0800
